In [1]:
import pandas as pd
import os
from tqdm import tqdm
import requests
import pandas as pd
import numpy as np
from datetime import datetime

from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem, DataStructs
from joblib import parallel_backend

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPool1D

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import pickle
import json

import imblearn

2024-12-10 20:10:07.120170: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-10 20:10:07.961260: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv("PubChem_bioassay_text_'progesterone receptor'.csv")
pubchem_json = json.load(open("PubChem_bioassay_text_progesterone receptor_records2.json"))
pubchem_dict = dict()
for assay in pubchem_json["PC_AssayContainer"]:
    pubchem_dict[assay["assay"]["descr"]["aid"]["id"]] = assay
df.aidname = df.aidname.str.lower()
df["type"] = "unknown"
df.loc[(df.aidname.str.contains("agonist")) & (~df.aidname.str.contains("antagonist")), "type"] = "agonist"
df.loc[(df.aidname.str.contains("antagonist")), "type"] = "antagonist"
df.loc[(df.aidname.str.contains("inhibition")), "type"] = "antagonist"
print(df.type.value_counts())
columns = ["PUBCHEM_RESULT_TAG", "PUBCHEM_SID", "PUBCHEM_CID", "PUBCHEM_EXT_DATASOURCE_SMILES", 
           "PUBCHEM_ACTIVITY_OUTCOME", "Standard Value", "Standard Units"]
columns = pd.Series(columns)
def find_target(row):
    aid = row["aid"]
    assay = pubchem_dict[aid]
    descr = assay["assay"]["descr"]
    if "target" in descr:
        targets = descr["target"]
        if len(targets) == 1 and targets[0]["name"].lower() == "progesterone receptor":
            row["target"] = "progesterone receptor"
            row["target_pr"] = True
        else:
            row["target"] = ",".join(t["name"] for t in targets)
            row["target_pr"] = False
    else:
        row["target"] = "unknown"
        row["target_pr"] = False
        
    return row

def contains_nm(row):
    aid = row["aid"]
    path = f"./PubChem_bioassay_text_'progesterone receptor'_records/AID_{aid}.csv"
    if os.path.getsize(path) == 0:
        row["contains_nm"] = False
        return row
    df = pd.read_csv(path, low_memory=False)

    if all(columns.isin(df.columns)) and (any(df["Standard Units"] == "nM") 
                                          or ("Potency-Replicate_1" in df.columns 
                                              and "Phenotype-Replicate_1" in df.columns)):
        row["contains_nm"] = True
    else:
        row["contains_nm"] = False
    
    return row
    

df = df.apply(lambda r: find_target(r), axis=1)
df = df.apply(lambda r: contains_nm(r), axis=1)
df["contains_expression"] = (df.aidname.str.contains("expression", case=False)  | df.aiddesc.str.contains("expression", case=False))
df["to_process"] = (~df["contains_expression"] & df["target_pr"] & df["contains_nm"])
df.to_csv("assays.csv")

type
unknown       1264
antagonist     645
agonist        199
Name: count, dtype: int64


In [3]:
files = os.listdir("./PubChem_bioassay_text_'progesterone receptor'_records")
assays = pd.read_csv("assays.csv")
assays.set_index("aid", drop=False, inplace=True)
print("before", assays.shape)
assays = assays[assays["to_process"]]
print("after", assays.shape)
print(assays.type.value_counts())

before (2108, 35)
after (278, 35)
type
unknown       130
antagonist    105
agonist        43
Name: count, dtype: int64


In [4]:
columns = ["PUBCHEM_RESULT_TAG", "PUBCHEM_SID", "PUBCHEM_CID", "PUBCHEM_EXT_DATASOURCE_SMILES", 
           "PUBCHEM_ACTIVITY_OUTCOME", "Standard Value", "Standard Units"]
columns = pd.Series(columns)

skipped = list()
dfs = list()
files = [f for f in files if ".csv" in f and "AID" in f]

for f in tqdm(files):
    aid = int(f.replace("AID_", "").replace(".csv", ""))
    special_aid = None
    if aid not in assays.aid.tolist():
        skipped.append(f)
        continue
    try:
        path = f"./PubChem_bioassay_text_'progesterone receptor'_records/{f}"
        df = pd.read_csv(path, low_memory=False)

        if "Potency-Replicate_1" in df.columns and "Phenotype-Replicate_1" in df.columns:
            special_aid = aid
            df.rename(inplace=True, columns={"Potency-Replicate_1": "Standard Value", "Phenotype-Replicate_1": "Label"})
            df["Standard Value"] = pd.to_numeric(df["Standard Value"], errors="coerce") * 1000 # to nM
            df["Standard Units"] = "nM"

        # remove header columns
        df["PUBCHEM_RESULT_TAG"] = pd.to_numeric(df["PUBCHEM_RESULT_TAG"], errors="coerce")
        df = df[~df["PUBCHEM_RESULT_TAG"].isna()]

        if all(columns.isin(df.columns)) and any(df["Standard Units"] == "nM"):
            df = df[columns]
            df = df[df["Standard Units"] == "nM"]
            df["AID"] = aid
            df["AID_TYPE"] = assays.loc[aid, "type"]
            df["Standard Value"] = df["Standard Value"].astype(float)
            df["Label"] = "Unknown"
            df.loc[(df["AID_TYPE"] == "agonist") & (df["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"), "Label"] = "Activator"
            df.loc[(df["AID_TYPE"] == "antagonist") & (df["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"), "Label"] = "Inhibitor"
            dfs.append(df)
        else:
            skipped.append(f)
            if special_aid is not None:
                print(f"WARN: special aid not processed {aid}")
    except pd.errors.EmptyDataError:
        pass

print("skipped", len(skipped), "of", len(files))

dataset = pd.concat(dfs, ignore_index=True)

100%|██████████| 2108/2108 [00:00<00:00, 2218.08it/s]

skipped 1830 of 2108


In [5]:
# 4 - Strong: Activity concentration  0-0.09 μM (0-90 nM).
# 3-  Moderate: Activity concentration between  0.09 and 0.18 μM (90 nM-180 nM).
# 2 - Weak: Activity concentration between 0.18 -  20 μM. (180 nM - 20 000 nM)
# 1 - Very Weak: Activity concentration between 20 - 800 μM. (20 000 nM - 800 000 nM)
# 0-  Inactive: Activity concentration higher than 800 μM or inactive 

LABELS = [("strong", 0, 90), 
          ("moderate", 90, 180), 
          ("weak", 180, 20000),
          ("very weak", 20000, 800000),
          ("inactive", 800000, None)
         ]

for label, r0, r1 in LABELS:
    if label != "inactive":
        count = dataset[(dataset["Standard Value"] > r0) & (dataset["Standard Value"] <= r1) 
                                & (dataset["PUBCHEM_ACTIVITY_OUTCOME"] == "Active")]
        print(f"{r0}-{r1}: {count.shape[0]}")
    else:
        count = dataset[(dataset["Standard Value"] > r0) | (dataset["PUBCHEM_ACTIVITY_OUTCOME"] == "Unspecified")]
        print(f"800000 or Unspecified Activity Outcome: {count.shape[0]}")

0-90: 1513
90-180: 189
180-20000: 940
20000-800000: 0
800000 or Unspecified Activity Outcome: 487


In [6]:
dataset.PUBCHEM_ACTIVITY_OUTCOME.value_counts()

PUBCHEM_ACTIVITY_OUTCOME
Active         2642
Unspecified     487
Inactive          1
Name: count, dtype: int64

In [7]:
dataset = dataset[dataset.PUBCHEM_ACTIVITY_OUTCOME.isin(["Active", "Inactive"])].copy()
dataset

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME,Standard Value,Standard Units,AID,AID_TYPE,Label
0,1.0,404718912.0,53359074.0,CCN(C)C(=O)C1=C(C=C(C=C1)C2=NC3=C(C=C2)[C@H](C...,Active,1829.0,nM,626800,unknown,Unknown
2,3.0,134449350.0,49851364.0,CC(C)([C@@H](C1=CC=CC=C1)C2=CC=C(C=C2)O)C(=O)N...,Active,273.0,nM,626800,unknown,Unknown
3,4.0,134460821.0,24952193.0,CC(C)([C@@H]1C2=C(C=C(C=C2)O)OC3=CC=CC=C13)C(=...,Active,53.0,nM,626800,unknown,Unknown
4,5.0,136920892.0,57390301.0,CC(C)OC1=CC=C(C=C1)C2=NC3=C(C=C2)[C@@H](C4=CC=...,Active,310.0,nM,626800,unknown,Unknown
5,6.0,136937246.0,57399054.0,CCCC1=CC=C(C=C1)C2=NC3=C(C=C2)[C@@H](C4=CC=CC=...,Active,285.0,nM,626800,unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...
3121,18.0,103387854.0,10383035.0,CC1CCN(N=C1C2=CC=CC=C2)C(=O)C3=CC(=C(C=C3)Cl)Cl,Active,24.0,nM,162111,unknown,Unknown
3122,19.0,103387871.0,10594721.0,CC1(CC(=NN(C1)C(=O)C2=CC(=C(C=C2)Cl)Cl)C3=CC=C...,Active,75.3,nM,162111,unknown,Unknown
3123,20.0,103387941.0,10641642.0,C1CC(=NN(C1)C(=O)C2=CC(=C(C=C2)Cl)Cl)C3=CC=C(C...,Active,48.2,nM,162111,unknown,Unknown
3125,22.0,103387972.0,3831396.0,C1CC(=NN(C1)C(=O)C2=CC(=C(C=C2)Cl)Cl)C3=CC=CC=C3,Active,62.4,nM,162111,unknown,Unknown


In [8]:
dataset.Label.value_counts()

Label
Unknown      1165
Inhibitor    1041
Activator     437
Name: count, dtype: int64

In [9]:
dataset["Group"] = "inactive"
for label, r0, r1 in LABELS:
    if label != "inactive":
        dataset.loc[(dataset["Standard Value"] > r0) & 
        (dataset["Standard Value"] <= r1) & 
        (dataset["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"), "Group"] = label

In [10]:
dataset = dataset[["PUBCHEM_CID", "PUBCHEM_EXT_DATASOURCE_SMILES", "Label", "Group", "AID"]]
dataset.columns = ["CID", "SMILES", "Label", "Group", "AID"]
dataset

,CID,SMILES,Label,Group,AID
0,53359074.0,CCN(C)C(=O)C1=C(C=C(C=C1)C2=NC3=C(C=C2)[C@H](C...,Unknown,weak,626800
2,49851364.0,CC(C)([C@@H](C1=CC=CC=C1)C2=CC=C(C=C2)O)C(=O)N...,Unknown,weak,626800
3,24952193.0,CC(C)([C@@H]1C2=C(C=C(C=C2)O)OC3=CC=CC=C13)C(=...,Unknown,strong,626800
4,57390301.0,CC(C)OC1=CC=C(C=C1)C2=NC3=C(C=C2)[C@@H](C4=CC=...,Unknown,weak,626800
5,57399054.0,CCCC1=CC=C(C=C1)C2=NC3=C(C=C2)[C@@H](C4=CC=CC=...,Unknown,weak,626800
...,...,...,...,...,...
3121,10383035.0,CC1CCN(N=C1C2=CC=CC=C2)C(=O)C3=CC(=C(C=C3)Cl)Cl,Unknown,strong,162111
3122,10594721.0,CC1(CC(=NN(C1)C(=O)C2=CC(=C(C=C2)Cl)Cl)C3=CC=C...,Unknown,strong,162111
3123,10641642.0,C1CC(=NN(C1)C(=O)C2=CC(=C(C=C2)Cl)Cl)C3=CC=C(C...,Unknown,strong,162111
3125,3831396.0,C1CC(=NN(C1)C(=O)C2=CC(=C(C=C2)Cl)Cl)C3=CC=CC=C3,Unknown,strong,162111


In [11]:
dataset.Group.value_counts()

Group
strong      1513
weak         940
moderate     189
inactive       1
Name: count, dtype: int64

In [12]:
# import requests

# for file in ["./Assay List TOX21_PR_BLA_Antagonist_ratio-2024-04-21.xlsx", "./Assay List TOX21_PR_BLA_Agonist_ratio-2024-04-21.xlsx"]:
#     df = pd.read_excel(file)
#     df["smiles"] = ""
#     for _, row in tqdm(df.iterrows(), total=df.shape[0]):
#         cas = str(row["CASRN"]).strip()
#         url = f"https://cactus.nci.nih.gov/chemical/structure/{cas}/smiles"
#         resp = requests.get(url)
#         if resp.status_code == 200:
#             smiles = resp.text
#             row["smiles"] = smiles

#     df.to_excel(file.replace(".xlsx", "_with_smiles.xlsx"))

In [13]:
# import requests
# file = "./Chemical list_experiments.csv"
# pubchem_file = pd.read_excel("./Pretraga po imenima.xlsx")
# df = pd.read_csv(file)
# df["smiles"] = ""
# for i, row in tqdm(df.iterrows(), total=df.shape[0]):
#     name = row["name"].strip()
#     url = f"https://api-ccte.epa.gov/chemical/search/contain/{name}"
#     resp = requests.get(url, headers={"x-api-key": "01cba3ec-904f-11ee-988e-325096b39f47", "accept": "application/json"})
#     j = resp.json()
#     smiles = None
#     if len(j) >= 1 and type(j) == list:
#         j_red = list(filter(lambda e: e["preferredName"] == name, j))
#         if len(j_red) == 1:
#             smiles = j_red[0]["smiles"]
#     if not smiles:
#         if name in pubchem_file["name"].tolist():
#             link = pubchem_file.loc[name == pubchem_file["name"], "link"].iloc[0]
#             cid = link.split("/")[-1]
            
#             smiles = pd.read_csv(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/CanonicalSMILES/CSV")["CanonicalSMILES"].iloc[0]
#             print(f"smiles from pubchem for {name}", smiles)
#     df.loc[i, "smiles"] = smiles
# df.dropna().to_csv(file.replace(".csv", "_with_smiles.csv"))

In [14]:
# for aeid in [2127, 2223, 2123, 2222]:
#     url = f"https://api-ccte.epa.gov/bioactivity/data/search/by-aeid/{aeid}"
#     resp = requests.get(url, headers={"x-api-key": "01cba3ec-904f-11ee-988e-325096b39f47", "accept": "application/hal+json"})
#     if resp.status_code == 200:
#         j = resp.json()
#         df = pd.DataFrame(j)
#         df.to_excel(f"assay_{aeid}.xlsx")


In [15]:
dfs = list()
print("before", dataset.shape)

for aeid, atype in zip([2127, 2223, 2123, 2222], ["Antagonist", "Antagonist", "Agonist", "Agonist"]):
    df = pd.read_csv(f"assay_{aeid}.csv")
    df_smiles = pd.read_csv(f"assay_{aeid}_smiles.csv")
    df["DTXSID"] = df["DTXSID"].str.replace("https://comptox.epa.gov/dashboard/chemical/details/", "")
    df = df.merge(df_smiles, on="DTXSID", how="inner")
    print(aeid, df.shape)
    df["AC50"] *= 1000
    df["Group"] = "inactive"
    for label, r0, r1 in LABELS:
        if label != "inactive":
            df.loc[(df["AC50"] > r0) & 
            (df["AC50"] <= r1) & 
            (df["HIT CALL"] == "Active"), "Group"] = label
    df["Label"] = "Unknown"
    df.loc[(df["HIT CALL"] == "Active"), "Label"] = "Activator" if atype == "Agonist" else "Inhibitor"
    df["AID"] = aeid
    df["CID"] = df["DTXSID"]
    df = df[['CID', 'SMILES', 'PREFERRED NAME', 'Label', 'Group', 'AID']]
    dfs.append((aeid, df.copy()))

for aeid, df in dfs:
    # train 2127 2123
    if aeid in (2127, 2123):
        del df["PREFERRED NAME"]
        dataset = pd.concat([dataset, df])
    else:
        df = df[['PREFERRED NAME', 'SMILES']]
        df.columns = ["name", "smiles"]
        df.to_csv(f"inference/{aeid}.csv")

print("after", dataset.shape)

before (2643, 5)
2127 (7871, 17)
2223 (99, 17)
2123 (7871, 17)
2222 (230, 17)
after (18385, 5)


In [16]:
new_dataset = list()
ds = dataset.copy()
for cid, grp in ds.groupby("CID"):
    # if cid == "DTXSID0048686":
    #     print(grp)
    #     break
    
    if len(grp) == 1:
        new_dataset.append(grp.iloc[0].to_dict())
    else:
        if any(grp.Label != "Unknown"):
            new_label = grp[grp.Label != "Unknown"]["Label"].mode().values[0]
        else:
            new_label = "Unknown"
        grp.Label = new_label

        if any(grp.Group != "inactive"):
            new_group = grp[grp.Group != "inactive"]["Group"].mode().values[0]
        else:
            new_group = "inactive"
        grp.Group = new_group

        new_dataset.append(grp.iloc[0].to_dict())
        print(f"{cid} new label", new_label, "new group", new_group)

print("before", dataset.shape)
dataset = pd.DataFrame(new_dataset)
print("after", dataset.shape)

5833.0 new label Inhibitor new group weak
5994.0 new label Activator new group strong
6230.0 new label Activator new group strong
6279.0 new label Activator new group strong
55245.0 new label Inhibitor new group strong
77098.0 new label Inhibitor new group moderate
119086.0 new label Unknown new group strong
1834356.0 new label Activator new group moderate
2255761.0 new label Inhibitor new group weak
2325104.0 new label Inhibitor new group weak
2876244.0 new label Inhibitor new group moderate
2941086.0 new label Activator new group weak
2952945.0 new label Activator new group weak
3160915.0 new label Activator new group weak
3775890.0 new label Unknown new group strong
3831396.0 new label Unknown new group strong
4111936.0 new label Inhibitor new group weak
4369524.0 new label Activator new group strong
5311505.0 new label Inhibitor new group strong
6439929.0 new label Unknown new group weak
6450278.0 new label Unknown new group weak
6918548.0 new label Inhibitor new group strong
95772

In [17]:
dataset["CID"].duplicated().sum()

0

In [18]:
dataset.to_csv("dataset.csv")

# Prepare for train

In [19]:
fingerprints = dict()
bad_smiles = list()

for index, row in tqdm(dataset.iterrows(), total=dataset.shape[0]):
    try:
        mol = Chem.MolFromSmiles(row["SMILES"])
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, arr)
        fingerprints[index] = arr
    except Exception as e:
        bad_smiles.append(index)

 91%|█████████ | 8669/9541 [00:02<00:00, 4168.44it/s][20:10:34] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[20:10:34] Explicit valence for atom # 6 O, 3, is greater than permitted
 95%|█████████▌| 9087/9541 [00:02<00:00, 4164.43it/s][20:10:34] Explicit valence for atom # 3 Si, 8, is greater than permitted
[20:10:34] Explicit valence for atom # 0 Cl, 2, is greater than permitted
100%|██████████| 9541/9541 [00:02<00:00, 4066.52it/s]


In [20]:
fingerprints[0][:100]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
len(fingerprints), len(bad_smiles)

(9253, 288)

In [22]:
dataset = dataset[~dataset.index.isin(bad_smiles)]
dataset.shape

(9253, 5)

# Train (no oversample, no agonist/antagonist)

In [ ]:
X = np.array(list(fingerprints.values()))
y = dataset.loc[fingerprints.keys(), "Group"].values
print(X.shape, y.shape)

In [ ]:
label_dict = {v: k for k, v in enumerate(np.unique(y))}
print(label_dict)

In [ ]:
y = keras.utils.to_categorical([label_dict[v] for v in y.tolist()], num_classes=len(label_dict))

In [ ]:
print(pd.DataFrame(y).value_counts())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(Conv1D(32, 16, activation='relu', input_shape=(2048, 1)))
model.add(MaxPool1D())
model.add(Conv1D(64, 32, activation='relu'))
model.add(MaxPool1D())
model.add(Conv1D(128, 16, activation='relu'))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=len(label_dict), activation='softmax'))

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'],
                  optimizer=tf.keras.optimizers.SGD(learning_rate=0.01))

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=3, monitor="val_accuracy"),
    keras.callbacks.ModelCheckpoint(filepath='models/model_no_oversample.{epoch:02d}-{val_loss:.2f}.keras', save_best_only=True),
    keras.callbacks.TensorBoard(log_dir=f'./tensorboard/no_oversample_{datetime.now()}'),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
]

In [ ]:
history = model.fit(
    X_train, y_train, epochs=25, verbose=1, validation_data=(X_test, y_test), callbacks=callbacks
)

In [ ]:
plt.plot(history.history["accuracy"], label='accuracy')
plt.plot(history.history["val_accuracy"], label='val_accuracy')
plt.plot(history.history["loss"], label='loss')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')

vacc = max(history.history["val_accuracy"])
acc = max(history.history["accuracy"])
print(acc, vacc)

In [ ]:
model.summary()

In [ ]:
preds = model.predict(X_test)
preds = np.argmax(tf.nn.softmax(preds), axis=1)
preds

In [ ]:
labels = np.argmax(y_test, axis=1)
labels

In [ ]:
print(classification_report(labels, preds, target_names=label_dict.keys()))

# Train (with oversample, no agonist/antagonist)

In [ ]:
X = np.array(list(fingerprints.values()))
y = dataset.loc[fingerprints.keys(), "Group"].values
print(X.shape, y.shape)

In [ ]:
label_dict = {v: k for k, v in enumerate(np.unique(y))}
print(label_dict)

In [ ]:
y = keras.utils.to_categorical([label_dict[v] for v in y.tolist()], num_classes=len(label_dict))

In [ ]:
print(pd.DataFrame(y).value_counts())

In [ ]:
print("start oversample", X.shape, y.shape)
    
with parallel_backend('threading', n_jobs=12):
    oversample = imblearn.over_sampling.SMOTE(k_neighbors=2)
    X_oversampled, y_oversampled = oversample.fit_resample(X, y)

X_oversampled = X_oversampled.reshape(-1, 2048, 1)
y_oversampled = y_oversampled.reshape(-1, len(label_dict))

print("end oversample", X_oversampled.shape, y_oversampled.shape)

X_train, X_test, y_train, y_test = train_test_split(X_oversampled, y_oversampled, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(Conv1D(32, 16, activation='relu', input_shape=(2048, 1)))
model.add(MaxPool1D())
model.add(Conv1D(64, 32, activation='relu'))
model.add(MaxPool1D())
model.add(Conv1D(128, 16, activation='relu'))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=len(label_dict), activation='softmax'))

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'],
                  optimizer=tf.keras.optimizers.SGD(learning_rate=0.01))

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=3, monitor="val_accuracy"),
    keras.callbacks.ModelCheckpoint(filepath='models/model_with_oversample.{epoch:02d}-{val_loss:.2f}.keras', save_best_only=True),
    keras.callbacks.TensorBoard(log_dir=f'./tensorboard/with_oversample_{datetime.now()}'),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
]

In [ ]:
model.save("model.keras")

In [ ]:
history = model.fit(
    X_train, y_train, epochs=25, verbose=1, validation_data=(X_test, y_test), callbacks=callbacks
)

In [ ]:
plt.plot(history.history["accuracy"], label='accuracy')
plt.plot(history.history["val_accuracy"], label='val_accuracy')
plt.plot(history.history["loss"], label='loss')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')

vacc = max(history.history["val_accuracy"])
acc = max(history.history["accuracy"])
print(acc, vacc)

In [ ]:
model.summary()

In [ ]:
preds = model.predict(X_test)
preds = np.argmax(tf.nn.softmax(preds), axis=1)
preds

In [ ]:
labels = np.argmax(y_test, axis=1)
labels

In [ ]:
print(classification_report(labels, preds, target_names=label_dict.keys()))

In [ ]:
model = keras.models.load_model("models/model_with_oversample.18-0.19.keras")

In [ ]:
for f in os.listdir("inference"):
    if ".csv" not in f:
        continue
    print(f)
    df = pd.read_csv(f"inference/{f}")
    if "Total Assays" in df.columns:
        df = df.loc[:, :"Total Assays"]
    fingerprints_inf = dict()
    bad_smiles = list()
    
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        try:
            # print(row["smiles"])
            mol = Chem.MolFromSmiles(row["smiles"])
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
            arr = np.zeros((1,))
            DataStructs.ConvertToNumpyArray(fp, arr)
            fingerprints_inf[index] = arr
        except Exception as e:
            bad_smiles.append(index)

    print("bad smiles", bad_smiles)
    df = df[~df.index.isin(bad_smiles)]
    X = np.array(list(fingerprints_inf.values())).reshape(-1, 2048, 1)

    preds = model.predict(X, verbose=True)
    preds = np.argmax(tf.nn.softmax(preds), axis=1)
    label_dict_inv = {v: k for k, v in label_dict.items()}
    preds = [label_dict_inv[v] for v in preds.tolist()]
    df["predictions_model1"] = preds
    df.to_csv(f"inference/{f}", index=False)


# Train (with oversample, with agonist/antagonist)

In [23]:
combined_labels = (dataset["Label"] + "_" + dataset["Group"])
combined_labels = pd.DataFrame({"Label": combined_labels.values}, index=combined_labels.index)
combined_labels = combined_labels.groupby("Label").filter(lambda grp: len(grp) > 1)
combined_labels.loc[combined_labels.Label == "Unknown_inactive", "Label"] = "Inactive"
combined_labels = combined_labels[~combined_labels.Label.str.contains("Unknown")]

In [24]:
combined_labels.value_counts()

Label              
Inactive               5543
Inhibitor_very weak    1137
Inhibitor_weak          991
Inhibitor_strong        387
Activator_strong        357
Inhibitor_moderate       88
Activator_weak           68
Activator_very weak      65
Activator_moderate       36
Name: count, dtype: int64

In [25]:
X = np.array([fingerprints.get(k) for k in combined_labels.index.tolist()])
y = np.array(combined_labels.Label.values.tolist())
print(X.shape, y.shape)

(8672, 2048) (8672,)


In [26]:
label_dict = {v: k for k, v in enumerate(np.unique(y))}
print(label_dict)

{'Activator_moderate': 0, 'Activator_strong': 1, 'Activator_very weak': 2, 'Activator_weak': 3, 'Inactive': 4, 'Inhibitor_moderate': 5, 'Inhibitor_strong': 6, 'Inhibitor_very weak': 7, 'Inhibitor_weak': 8}


In [27]:
y = keras.utils.to_categorical([label_dict[v] for v in y.tolist()], num_classes=len(label_dict))

In [28]:
print(pd.DataFrame(y).value_counts())

0    1    2    3    4    5    6    7    8  
0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0    5543
                    0.0  0.0  0.0  1.0  0.0    1137
                                   0.0  1.0     991
                              1.0  0.0  0.0     387
     1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     357
     0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0      88
               1.0  0.0  0.0  0.0  0.0  0.0      68
          1.0  0.0  0.0  0.0  0.0  0.0  0.0      65
1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      36
Name: count, dtype: int64


In [29]:
print("start oversample", X.shape, y.shape)
    
with parallel_backend('threading', n_jobs=12):
    oversample = imblearn.over_sampling.SMOTE(k_neighbors=2)
    X_oversampled, y_oversampled = oversample.fit_resample(X, y)

X_oversampled = X_oversampled.reshape(-1, 2048, 1)
y_oversampled = y_oversampled.reshape(-1, len(label_dict))

print("end oversample", X_oversampled.shape, y_oversampled.shape)

X_train, X_test, y_train, y_test = train_test_split(X_oversampled, y_oversampled, test_size=0.2, random_state=42)

start oversample (8672, 2048) (8672, 9)
end oversample (49887, 2048, 1) (49887, 9)


In [30]:
model = Sequential()
model.add(Conv1D(32, 16, activation='relu', input_shape=(2048, 1)))
model.add(MaxPool1D())
model.add(Conv1D(64, 32, activation='relu'))
model.add(MaxPool1D())
model.add(Conv1D(128, 16, activation='relu'))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=len(label_dict), activation='softmax'))

/home/milosevicn/miniconda3/envs/bio/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
2024-12-10 20:10:36.259878: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-10 20:10:36.293096: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-10 2

In [31]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'],
                  optimizer=tf.keras.optimizers.SGD(learning_rate=0.01))

In [32]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=10, monitor="val_accuracy"),
    keras.callbacks.ModelCheckpoint(filepath='models/{epoch:03d}_model_with_oversample_ago.{val_loss:.2f}.weights.h5', 
                                    save_best_only=True,
                                    save_weights_only=True),
    keras.callbacks.TensorBoard(log_dir=f'./tensorboard/with_oversample_agonist_antagonist_{datetime.now()}'),
    keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=3, min_lr=0.0001)
]

In [ ]:
history = model.fit(
    X_train, y_train, epochs=50, verbose=1, validation_data=(X_test, y_test), callbacks=callbacks
)

In [ ]:
plt.plot(history.history["accuracy"], label='accuracy')
plt.plot(history.history["val_accuracy"], label='val_accuracy')
plt.plot(history.history["loss"], label='loss')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')

vacc = max(history.history["val_accuracy"])
acc = max(history.history["accuracy"])
print(acc, vacc)

In [33]:
# !pip install plotly_express
import plotly_express as px

hist = pd.DataFrame(history.history)
hist["epoch"] = hist.index

px.line(hist, x="epoch", y=["accuracy", "val_accuracy", "loss"], 
        labels={'epoch': 'epoch', 'value':'', 'variable': ''})

NameError: name 'history' is not defined

In [34]:
model.load_weights("models/model_with_oversample_ago.13-0.08.weights.h5")
model.summary()

2024-12-10 20:10:45.297731: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 250085376 exceeds 10% of free system memory.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 2033, 32)       │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1016, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 985, 64)        │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 492, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 477, 128)       │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 61056)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    62,522,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 9)              │         4,617 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 63,249,129 (241.28 MB)

 Trainable params: 63,249,129 (241.28 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
preds = model.predict(X_test)
preds = np.argmax(tf.nn.softmax(preds), axis=1)
preds

2024-12-10 20:10:45.853422: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 81739776 exceeds 10% of free system memory.
2024-12-10 20:10:45.908474: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 81739776 exceeds 10% of free system memory.
I0000 00:00:1733857846.022179  425358 service.cc:145] XLA service 0x7f6f50da4e20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733857846.022206  425358 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 SUPER, Compute Capability 7.5
2024-12-10 20:10:46.030286: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-10 20:10:46.067349: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


105/312 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

I0000 00:00:1733857846.565227  425358 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([8, 3, 8, ..., 4, 7, 1])

In [36]:
labels = np.argmax(y_test, axis=1)
labels

array([8, 3, 8, ..., 4, 7, 1])

In [37]:
print(classification_report(labels, preds, target_names=label_dict.keys()))

                     precision    recall  f1-score   support

 Activator_moderate       1.00      1.00      1.00      1091
   Activator_strong       1.00      1.00      1.00      1099
Activator_very weak       0.99      1.00      1.00      1034
     Activator_weak       0.99      1.00      1.00      1136
           Inactive       0.95      0.89      0.92      1140
 Inhibitor_moderate       1.00      1.00      1.00      1140
   Inhibitor_strong       0.99      0.99      0.99      1091
Inhibitor_very weak       0.91      0.95      0.93      1109
     Inhibitor_weak       0.97      0.97      0.97      1138

           accuracy                           0.98      9978
          macro avg       0.98      0.98      0.98      9978
       weighted avg       0.98      0.98      0.98      9978



In [ ]:
dfs = []
os.system("rm inference/all.csv")
for f in os.listdir("inference"):
    if ".csv" not in f:
        continue
    df = pd.read_csv(f"inference/{f}")
    fingerprints_inf = dict()
    bad_smiles = list()
    
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        try:
            mol = Chem.MolFromSmiles(row["smiles"])
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
            arr = np.zeros((1,))
            DataStructs.ConvertToNumpyArray(fp, arr)
            fingerprints_inf[index] = arr
        except Exception as e:
            bad_smiles.append(index)

    print("bad smiles", bad_smiles)
    df = df[~df.index.isin(bad_smiles)]
    X = np.array(list(fingerprints_inf.values())).reshape(-1, 2048, 1)

    preds = model.predict(X, verbose=True)
    preds = np.argmax(tf.nn.softmax(preds), axis=1)
    label_dict_inv = {v: k for k, v in label_dict.items()}
    preds = [label_dict_inv[v] for v in preds.tolist()]
    df["predictions_model2"] = preds
    df["filename"] = f
    df["substance_in_dataset"] = df["smiles"].isin(dataset["SMILES"])
    df["substance_experimental"] = "Unknown"
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        if row["substance_in_dataset"]:
            dss = dataset[dataset.SMILES == row["smiles"]].iloc[0]
            df.loc[index, "substance_experimental"] = dss["Label"] + "_" + dss["Group"]
    #df.to_csv(f"inference/{f}", index=False)
    dfs.append(df)

df_all = pd.concat(dfs)
df_all.to_csv("inference/all.csv")

In [ ]:
df_all

In [ ]:
experimental = df_all[(df_all.substance_in_dataset)].copy()
print(experimental.shape[0])

experimental.loc[experimental.substance_experimental.str.contains("inactive", case=False), "substance_experimental"] = "inactive"

correct = (experimental["predictions_model2"].str.lower() == experimental["substance_experimental"].str.lower()).sum()

print(correct)
print(correct / experimental.shape[0])

In [ ]:
df_all.predictions_model2.value_counts()

In [ ]:
experimental.substance_experimental.value_counts()

In [ ]:
experimental.predictions_model2.value_counts()

In [ ]:
new_detections = df_all[(~df_all.substance_in_dataset) & (df_all.predictions_model2.str.contains("strong"))].copy()

In [ ]:
del new_detections["Unnamed: 0"]
new_detections.loc[new_detections.name.isna(), "name"] = new_detections.loc[new_detections.name.isna(), "PREFERRED NAME"]

In [ ]:
new_detections

In [ ]:
new_detections.to_csv("new_detections.csv")

In [ ]:
df_all[df_all.DTXSID == "DTXSID8051706"]

In [ ]:
df_all

In [39]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
sns.set_theme(rc={'figure.figsize':(11.7,8.27)})
import matplotlib.pyplot as plt     
import plotly_express as px
import pandas as pd

df_all = pd.read_csv("inference/all.csv")
df = df_all.copy()
df = df[df.substance_in_dataset]
df = df[["name", "PREFERRED NAME", "predictions_model2", "substance_experimental"]]
df.columns = ["name", "name_alt", "prediction", "experimental"]
df.experimental = df.experimental.replace("Unknown_inactive", "Inactive")

print((df.experimental == df.prediction).sum() / len(df))

cm = confusion_matrix(df.experimental, df.prediction)
cm = pd.DataFrame(cm, columns=label_dict.keys(), index=label_dict.keys())
cm.to_csv("conf_matrix_1.csv")
px.imshow(cm, text_auto=True, aspect="auto").write_html("conf_matrix_1.html")
px.imshow(cm, text_auto=True, aspect="auto").write_image("conf_matrix_1.pdf", format="pdf")

ranges = sorted(["Activator", "Inactive", "Inhibitor"])
for name in ranges:
    df.loc[df.prediction.str.contains(name), "prediction"] = name
    df.loc[df.experimental.str.contains(name), "experimental"] = name

cm = confusion_matrix(df.experimental, df.prediction)
cm = pd.DataFrame(cm, columns=["Agonist", "Inactive", "Antagonist"], index=["Agonist", "Inactive", "Antagonist"])
cm.to_csv("conf_matrix_2.csv")
px.imshow(cm, text_auto=True, aspect="auto").write_html("conf_matrix_2.html")
px.imshow(cm, text_auto=True, aspect="auto").write_image("conf_matrix_2.pdf", format="pdf")

df = df_all.copy()
df = df[df.substance_in_dataset]
df = df[["name", "PREFERRED NAME", "predictions_model2", "substance_experimental"]]
df.columns = ["name", "name_alt", "prediction", "experimental"]
df.experimental = df.experimental.replace("Unknown_inactive", "Inactive")

df.prediction = df.prediction.str.split("_", expand=True)[1]
df.experimental = df.experimental.str.split("_", expand=True)[1]

df.fillna("Inactive", inplace=True)

ranges = sorted(df.experimental.unique().tolist())

cm = confusion_matrix(df.experimental, df.prediction)
cm = pd.DataFrame(cm, columns=ranges, index=ranges)
cm.to_csv("conf_matrix_3.csv")
px.imshow(cm, text_auto=True, aspect="auto").write_html("conf_matrix_3.html")
px.imshow(cm, text_auto=True, aspect="auto").write_image("conf_matrix_3.pdf", format="pdf")

0.972042634981653
